<a href="https://colab.research.google.com/github/benihime91/pytorch_retinanet/blob/master/nbs/debugging_cuda_error.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/benihime91/pytorch_retinanet.git # Clone the Repo

Cloning into 'pytorch_retinanet'...
remote: Enumerating objects: 276, done.
remote: Counting objects: 100% (276/276), done.
remote: Compressing objects: 100% (195/195), done.
remote: Total 722 (delta 191), reused 154 (delta 81), pack-reused 446
Receiving objects: 100% (722/722), 4.25 MiB | 10.19 MiB/s, done.
Resolving deltas: 100% (466/466), done.


In [2]:
# Extracting the Data assuming data is stored in `/content/drive/My Drive/Data/oxford-iiit-pet.tgz`
! tar zxf /content/drive/My\ Drive/Data/oxford-iiit-pet.tgz -C /content/ 
! echo " >> Data Unziped"

 >> Data Unziped


In [3]:
%%bash
# Colab Setup : Install Required Libraries
pip install -U pytorch-lightning --quiet
pip install -U git+https://github.com/albumentations-team/albumentations --quiet
pip install -U bounding_box --quiet
echo " >> Libraries successfully installed !"

 >> Libraries successfully installed !


ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 2.2.0 which is incompatible.


In [5]:
import os
os.chdir('/content/pytorch_retinanet')

In [7]:
! CUDA_LAUNCH_BLOCKING=1 python /content/pytorch_retinanet/cuda_error_decode.py

2020-08-22 20:38:03.836999: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
100% 44.7M/44.7M [00:00<00:00, 274MB/s]
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type      | Params
------------------------------------
0 | model | Retinanet | 20 M  
creating index...
index created!
Validation sanity check: 100% 1/1.0 [00:01<00:00,  1.10s/it]Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.00s).
Accumulating evaluation results...
DONE (t=0.02s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 